In [11]:
x = rand(UInt8, 8 * 10);

In [12]:
f(a,b,c,d,e,f,g,h) = (UInt64(a) << 56) | (UInt64(b) << 48) | 
    (UInt64(c) << 40) | (UInt64(d) << 32) | 
    (UInt64(e) << 24) | (UInt64(f) << 16) | 
    (UInt64(g) << 8) | UInt64(h)

f (generic function with 1 method)

In [13]:
f(x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8])

0x0b05bbe66617b931

In [15]:
x[1:8]'

1×8 RowVector{UInt8,Array{UInt8,1}}:
 0x0b  0x05  0xbb  0xe6  0x66  0x17  0xb9  0x31

In [16]:
f.(x[1:8:end],x[2:8:end],x[3:8:end],x[4:8:end],
    x[5:8:end], x[6:8:end], x[7:8:end], x[8:8:end])

10-element Array{UInt64,1}:
 0x0b05bbe66617b931
 0xafb8d712044e7181
 0xdb0ffeb14b74c011
 0x7fa438ef45da8f3e
 0x28a8bdced1256314
 0x0f1d1c0cdbbefc70
 0xf51764e4a58656da
 0x891ec8f6c47b5943
 0x8583083c706b5759
 0x7470306da327d5f5

In [17]:
N = 10000
x = rand(UInt8, 8 * N);

In [18]:
length(x)

80000

In [37]:
display(x[1:8]')
display(x[end-7:end]')

1×8 RowVector{UInt8,Array{UInt8,1}}:
 0xc9  0xac  0xb8  0x31  0x44  0xfc  0xd2  0xcd

1×8 RowVector{UInt8,Array{UInt8,1}}:
 0xf2  0x77  0xb9  0x60  0xfc  0x3f  0x0c  0x07

In [19]:
using BenchmarkTools

In [34]:
@btime f.(x[1:8:end],x[2:8:end],x[3:8:end],x[4:8:end],
    x[5:8:end], x[6:8:end], x[7:8:end], x[8:8:end])

  230.551 μs (157 allocations: 161.95 KiB)


10000-element Array{UInt64,1}:
 0xc9acb83144fcd2cd
 0xc4f42255e8d20df7
 0xec5f8c5d48dc1a26
 0x0e31627649247cfb
 0x9cf26cd769fc0efa
 0x8e9ee322db7c3149
 0x05f29fc6c487d387
 0x94d59f0518fe00cd
 0x41049aad3c43502f
 0x67af83f5a29997e2
 0x581afcc91b25d385
 0xf592270c6f064111
 0x26fe03a6950dc94e
                  ⋮
 0x675692b97a8a42a9
 0x6909014f1e011b4d
 0x374dcdd67ae7f908
 0x0e019ba1cd4d848b
 0xd60fdd304b8fb712
 0x791b5b0cabcc49c7
 0x92eebc29bc31673b
 0x0d74c03df1e080aa
 0xff766c9db369cde3
 0xe6a8fcb1fb31dbfe
 0xb474931908d8e344
 0xf277b960fc3f0c07

In [41]:
A = Array{UInt64}(N)
z = 1
@btime for i in 1:8:N 
    A[z] = bswap.(reinterpret(UInt64, x[i:i+8-1]))[1]
end
A

  13.950 ms (44748 allocations: 1.79 MiB)


10000-element Array{UInt64,1}:
 0x5e7a91c05db5fa0a
 0x0000000000000000
 0x00007f9616dbd000
 0x0000000000000000
 0x00007f9616dbd038
 0x00007f9616dbd048
 0x00007f9616dbd078
 0x00007f9616dbd126
 0x0000000100000000
 0x00007f9616dbd120
 0x0000000100000000
 0x00007f9616dbd240
 0x0000000100000000
                  ⋮
 0x0000000000000001
 0x0000000000000000
 0x0000000000000000
 0x00007f9616dd07b0
 0x00007f9616dd06f0
 0x00007f9616dd0870
 0x0000000000000001
 0x00007f9616dd07b0
 0xffffffffffffff58
 0x0000000000000000
 0x0000000000000000
 0x00007f9616dd07b0

In [43]:
display(x[1:8]')
bswap.(reinterpret(UInt64, x[1:8]))

1×8 RowVector{UInt8,Array{UInt8,1}}:
 0xc9  0xac  0xb8  0x31  0x44  0xfc  0xd2  0xcd

1-element Array{UInt64,1}:
 0xc9acb83144fcd2cd

In [88]:
N = 1_000_000
x = rand(UInt8, 8 * N);

In [174]:
# orig implementation
function convertfloat(bytes)
    values = [bytes[i:i+8-1] for i in 1:8:length(bytes)]
    convertedvalues = map(x -> reinterpret(Float64, x)[1], values)
    convertedvalues
end
@time convertfloat(x)

 14.248540 seconds (3.02 M allocations: 184.288 MiB, 94.52% gc time)


1000000-element Array{Float64,1}:
  1.00233e145 
 -2.59206e-130
 -5.15168e-256
  1.04767e47  
  1.5293e122  
  3.27014e256 
 -1.61632e-133
 -8.81354e293 
  1.37967e-22 
 -1.0807e-61  
 -1.34795e-203
  5.34665e-119
 -1.7347e-216 
  ⋮           
  4.17391e-86 
  1.98012e-236
  5.06044e8   
  1.10956e281 
  7.88792e272 
  2.02462e-95 
  1.44653e48  
 -1.55767e234 
  4.79482e-52 
  8.47259e28  
  2.53801e266 
  3.60777e104 

In [145]:
reinterpretF64(v) = reinterpret(Float64, v)
function convertfloat(bytes)
    values = [bytes[i:i+8-1] for i in 1:8:length(bytes)]
    reinterpretF64.(values)
end
@time convertfloat(x)

  9.837314 seconds (3.03 M allocations: 184.436 MiB, 86.17% gc time)


1000000-element Array{Array{Float64,1},1}:
 [1.00233e145]  
 [-2.59206e-130]
 [-5.15168e-256]
 [1.04767e47]   
 [1.5293e122]   
 [3.27014e256]  
 [-1.61632e-133]
 [-8.81354e293] 
 [1.37967e-22]  
 [-1.0807e-61]  
 [-1.34795e-203]
 [5.34665e-119] 
 [-1.7347e-216] 
 ⋮              
 [4.17391e-86]  
 [1.98012e-236] 
 [5.06044e8]    
 [1.10956e281]  
 [7.88792e272]  
 [2.02462e-95]  
 [1.44653e48]   
 [-1.55767e234] 
 [4.79482e-52]  
 [8.47259e28]   
 [2.53801e266]  
 [3.60777e104]  

In [101]:
reinterpretF64(v) = reinterpret(Float64, v)
v = rand(UInt64, 10_000)
@btime reinterpret.(Float64, $(v));
@btime reinterpretF64.($(v));

  7.080 μs (2 allocations: 78.20 KiB)
  7.109 μs (2 allocations: 78.20 KiB)


In [104]:
v = rand(UInt8, 8 * 4);
v'

1×32 RowVector{UInt8,Array{UInt8,1}}:
 0x10  0xf4  0xac  0x8a  0xb6  0x6d  …  0x62  0x1e  0xda  0xbf  0xc7  0x75

In [120]:
w = [v[i:i+8-1] for i in 1:8:length(v)]

4-element Array{Array{UInt8,1},1}:
 UInt8[0x10, 0xf4, 0xac, 0x8a, 0xb6, 0x6d, 0xff, 0xbe]
 UInt8[0x69, 0xd2, 0xe4, 0x85, 0x8d, 0xe1, 0x05, 0xe9]
 UInt8[0xfc, 0xcd, 0x4a, 0xda, 0xb3, 0x38, 0xff, 0xd7]
 UInt8[0x60, 0xdf, 0x62, 0x1e, 0xda, 0xbf, 0xc7, 0x75]

In [124]:
w = UInt64.(reshape(v, (8,4)))'

4×8 Array{UInt64,2}:
 0x0000000000000010  0x00000000000000f4  …  0x00000000000000be
 0x0000000000000069  0x00000000000000d2     0x00000000000000e9
 0x00000000000000fc  0x00000000000000cd     0x00000000000000d7
 0x0000000000000060  0x00000000000000df     0x0000000000000075

In [125]:
map(x -> x & 0xf0, w)

4×8 Array{UInt64,2}:
 0x0000000000000010  0x00000000000000f0  …  0x00000000000000b0
 0x0000000000000060  0x00000000000000d0     0x00000000000000e0
 0x00000000000000f0  0x00000000000000c0     0x00000000000000d0
 0x0000000000000060  0x00000000000000d0     0x0000000000000070

In [153]:
function convertfloat(v) 
    w = 
        map(x -> x << 56, UInt64.(view(v, 1:8:length(v)))) +
        map(x -> x << 48, UInt64.(view(v, 2:8:length(v)))) +
        map(x -> x << 40, UInt64.(view(v, 3:8:length(v)))) +
        map(x -> x << 32, UInt64.(view(v, 4:8:length(v)))) +
        map(x -> x << 24, UInt64.(view(v, 5:8:length(v)))) +
        map(x -> x << 16, UInt64.(view(v, 6:8:length(v)))) +
        map(x -> x << 8, UInt64.(view(v, 7:8:length(v)))) +
        UInt64.(view(v, 8:8:length(v)))
    reinterpret.(Float64, w)
end
@time convertfloat(x)

 15.748656 seconds (133.47 k allocations: 181.569 MiB, 95.27% gc time)


1000000-element Array{Float64,1}:
  4.41886e160 
  2.38418e44  
 -2.26027e-159
 -3.01546e212 
  1.57731e99  
  5.64841e-64 
 -4.70443e-70 
 -6.6782e-301 
  1.9059e-66  
  2.0097e-61  
 -1.80226e-293
 -5.35467e-46 
  2.85897e44  
  ⋮           
 -6.91251e92  
  1.1118e114  
  8.1621e-245 
 -2.88532e19  
 -4.0852e147  
 -2.11412e-216
 -1.63707e-43 
 -2.00781e-223
 -2.96384e294 
 -1.09147e291 
 -9.43421e286 
  4.29114e280 

In [179]:
f(a,b,c,d,e,f,g,h) = (UInt64(a) << 56) | (UInt64(b) << 48) | 
    (UInt64(c) << 40) | (UInt64(d) << 32) | 
    (UInt64(e) << 24) | (UInt64(f) << 16) | 
    (UInt64(g) << 8) | UInt64(h)
function convertfloat(x) 
    reinterpret.(Float64,
        f.(x[1:8:end],x[2:8:end],x[3:8:end],x[4:8:end],
            x[5:8:end], x[6:8:end], x[7:8:end], x[8:8:end]))
end
function convertfloatlittle(x) 
    v = reverse(x)
    r = reinterpret.(Float64,
        f.(x[1:8:end],x[2:8:end],x[3:8:end],x[4:8:end],
            x[5:8:end], x[6:8:end], x[7:8:end], x[8:8:end]))
    reverse(r)
end
function convertfloat_orig(bytes)
    values = [bytes[i:i+8-1] for i in 1:8:length(bytes)]
    convertedvalues = map(x -> reinterpret(Float64, x)[1], values)
    convertedvalues
end

convertfloat_orig (generic function with 1 method)

In [180]:
@time r = convertfloat_orig(x);
println(r[1:15])
@time r =convertfloatlittle(x);
r[1:5]

 29.274079 seconds (3.02 M allocations: 184.297 MiB, 96.31% gc time)
[1.00233e145, -2.59206e-130, -5.15168e-256, 1.04767e47, 1.5293e122, 3.27014e256, -1.61632e-133, -8.81354e293, 1.37967e-22, -1.0807e-61, -1.34795e-203, 5.34665e-119, -1.7347e-216, 1.28032e-11, 1.06889e217]
  0.378573 seconds (94.15 k allocations: 34.231 MiB, 5.68% gc time)


5-element Array{Float64,1}:
  4.29114e280 
 -9.43421e286 
 -1.09147e291 
 -2.96384e294 
 -2.00781e-223

In [181]:
# Version a.  Original implementation... slow.
"""
Byte swap is needed only if file the array represent a different endianness
than the system.  This function does not make any assumption and the caller
is expected to pass `true` to the `swap` argument when needed.
"""
function convertfloat64a(bytes::Vector{UInt8}, swap::Bool)
    values = [bytes[i:i+8-1] for i in 1:8:length(bytes)]
    values = map(x -> reinterpret(Float64, x)[1], values)
    swap ? bswap.(values) : values
end

# Version b.  Should be a lot faster.  
"""
It turns out that `reinterpret` consider a single UInt64 as BigEndian 
Hence it's necessary to swap bytes if the array is in LittleEndian convention.
This function does not make any assumption and the caller
is expected to pass `true` to the `swap` argument when needed.
"""
function convertfloat64b(bytes::Vector{UInt8}, endianess::Symbol) 
    v = endianess == :LittleEndian ? reverse(bytes) : bytes
    c = convertint64.(v[1:8:end],v[2:8:end],v[3:8:end],v[4:8:end],
            v[5:8:end], v[6:8:end], v[7:8:end], v[8:8:end])
    r = reinterpret.(Float64, c)
    endianess == :LittleEndian ? reverse(r) : r
end

"""
Take 8 bytes and convert them into a UInt64 type.  The order is preserved.
"""
function convertint64(a::UInt8,b::UInt8,c::UInt8,d::UInt8,e::UInt8,f::UInt8,g::UInt8,h::UInt8)
    (UInt64(a) << 56) | (UInt64(b) << 48) | 
    (UInt64(c) << 40) | (UInt64(d) << 32) | 
    (UInt64(e) << 24) | (UInt64(f) << 16) | 
    (UInt64(g) << 8)  |  UInt64(h)
end

convertint64

In [184]:
@time z1 = convertfloat64a(x, false)
display(z1[1:3])

3-element Array{Float64,1}:
  1.00233e145 
 -2.59206e-130
 -5.15168e-256

 21.060591 seconds (3.00 M allocations: 183.106 MiB, 96.06% gc time)


In [185]:
@time z2 = convertfloat64b(x, :LittleEndian)
display(z2[1:3])

3-element Array{Float64,1}:
  1.00233e145 
 -2.59206e-130
 -5.15168e-256

  0.289733 seconds (44 allocations: 38.149 MiB, 16.56% gc time)
